In [ ]:
# Upload datasets
from google.colab import files
uploaded_files = files.upload()  # Upload salary_train.csv and salary_test.csv

# Import necessary packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Read training and test data
train_df = pd.read_csv("salary_train.csv")
test_df = pd.read_csv("salary_test.csv")
print(f"Train data: {train_df.shape} | Test data: {test_df.shape}")

# Add source column for later split
train_df["dataset"] = "train"
test_df["dataset"] = "test"
test_df["salary"] = np.nan  # Needed for alignment

# Combine for unified preprocessing
full_data = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

# Fill missing values
for column in full_data.columns:
    if full_data[column].dtype == "object":
        full_data[column] = full_data[column].fillna("NotAvailable")
    else:
        full_data[column] = full_data[column].fillna(full_data[column].median())

# Encode all categorical features
label_encoders = {}
object_columns = full_data.select_dtypes(include="object").columns

for feature in object_columns:
    encoder = LabelEncoder()
    full_data[feature] = encoder.fit_transform(full_data[feature].astype(str))
    label_encoders[feature] = encoder

# Separate the combined data back
train_set = full_data[full_data["dataset"] == label_encoders["dataset"].transform(["train"])[0]].drop(columns=["dataset"])
test_set = full_data[full_data["dataset"] == label_encoders["dataset"].transform(["test"])[0]].drop(columns=["dataset", "salary"])

X_train = train_set.drop("salary", axis=1)
y_train = train_set["salary"]

# Define and train ensemble model
model_rf = RandomForestRegressor(random_state=42)
model_gb = GradientBoostingRegressor(random_state=42)
voting_model = VotingRegressor(estimators=[("rf_model", model_rf), ("gb_model", model_gb)])
voting_model.fit(X_train, y_train)

# Ensure test set is not empty before predicting
if test_set.shape[0] == 0:
    raise ValueError("⚠️ Test data is empty. Check the contents of 'salary_test.csv'.")

# Generate predictions and save to file
test_predictions = voting_model.predict(test_set)
output_df = pd.DataFrame({"PredictedSalary": test_predictions})
output_df.to_csv("predicted_salaries.csv", index=False)

print("📁 Output file 'predicted_salaries.csv' generated successfully.")
files.download("predicted_salaries.csv")
